In [ ]:
%matplotlib widget

import numpy as np
import yaml
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
uploader = widgets.FileUpload(accept='.yml', multiple=False)
display(uploader)

In [ ]:
calib, = yaml.safe_load(uploader.value[0].content.tobytes())
camera = calib['camera']
focus = camera['focus']
sx = camera['sx']
sy = camera['sy']
cx = camera['cx']
cy = camera['cy']
k1 = camera['k1']
k2 = camera['k2']
k3 = camera['k3']
p1 = camera['p1']
p2 = camera['p2']
image_width = camera['image_width']
image_height = camera['image_height']

observ_points = [np.array(p) for p in calib['calib_obj']['observ_points']]
object_points = [np.array(p) for p in calib['calib_obj']['object_points']]
reproject_points = [np.array(p) for p in calib['calib_obj']['reproject_points']]
rvecs = np.array(calib['calib_obj']['rvecs'])
tvecs = np.array(calib['calib_obj']['tvecs'])
diff_points = [observ_point - reproject_point for observ_point, reproject_point in zip(observ_points, reproject_points)]

npos = len(object_points)
ylim_min, ylim_max = np.min(diff_points), np.max(diff_points)
xlim_max = np.max([p.shape[0] for p in object_points])

In [ ]:
def draw(pos):
    fig = plt.figure(0)
    if not fig.axes:
        ax1 = plt.subplot(1, 2, 1)
        ax2 = plt.subplot(2, 2, 2)
        ax3 = plt.subplot(2, 2, 4, sharex=ax2, sharey=ax2)

        ax1.plot([], [], 'b.', label='observ points')
        ax1.plot([], [], 'rx', label='reproject points')
        ax1.set_xlim(0, image_width)
        ax1.set_ylim(image_height, 0)
        ax1.set_xlabel('column')
        ax1.set_ylabel('row')
        ax1.legend()

        ax2.set_title('Deviation in column')
        ax2.plot([], 'b.')
        ax2.set_xlim(0, xlim_max - 1)
        ax2.set_ylim(ylim_min, ylim_max)
        ax2.set_xlabel('index')
        ax2.set_ylabel('pixel')

        ax3.set_title('Deviation in row')
        ax3.plot([], 'b.')
        ax3.set_xlabel('index')
        ax3.set_ylabel('pixel')
    else:
        ax1, ax2, ax3 = fig.axes

    ax1.set_title
    ln1, ln2 = ax1.get_lines()
    ln1.set_data(observ_points[pos][..., 0], observ_points[pos][..., 1])
    ln2.set_data(reproject_points[pos][..., 0], reproject_points[pos][..., 1])

    ln3, = ax2.get_lines()
    ln3.set_data(range(len(diff_points[pos][..., 0])), diff_points[pos][..., 0])

    ln4, = ax3.get_lines()
    ln4.set_data(range(len(diff_points[pos][..., 1])), diff_points[pos][..., 1])
    plt.show()

In [ ]:
interact(draw, pos=(0, npos - 1))